In [1]:
using ProgressBars

In [2]:
mutable struct Colony
    ants
    size::Int64
end

In [3]:
mutable struct Ant
    tour
    fitness::Float64
end

In [4]:
struct Node
    x::Float64
    y::Float64
end

In [5]:
function create_graph(n)
    println(n)
    x,y=[],[]
    #4
    if n==Int64(1)
        x = [ 0.09 , 0.16 , 0.84 , 0.70 ]
        y = [0.17, 0.52, 0.92, 0.16]   
    end
    if n==2
        x = [16.47000,16.47000,20.09000,22.39000,25.23000,22,20.47000,17.20000,16.30000,14.05000,16.53000,21.52000,19.41000,20.09000]
        y = [96.10000,94.44000,92.54000,93.37000,97.24000,96.05000,97.02000,96.29000,97.38000,98.12000,97.38000,95.59000,97.13000,94.55000]
    end
    # 50
    if n==3
        x = [7,9,2,0,0,1,7,1,2,51,42,31,5,12,36,52,27,17,13,57,62,42,16,8,7,27,30,43,58,58,37,38,46,61,62,63,32,45,59,5,10,21,5,30,39,32,25,25,48,56]
        y = [52,49,64,26,30,47,63,62,33,21,41,32,25,42,16,41,23,33,13,58,42,57,57,52,38,68,48,67,48,27,69,46,10,33,63,69,22,35,15,6,17,10,64,15,10,39,32,55,28,37]
    end
    #100
    if n==4
        x= [1380,2848,3510,457,3888,984,2721,1286,2716,738,1251,2728,3815,3683,1247,123,1234,252,611,2576,928,53,1807,274,2574,178,2678,1795,3384,3520,1256,1424,3913,3085,2573,463,3875,298,3479,2542,3955,1323,3447,2936,1621,3373,1393,3874,938,3022,2482,3854,376,2519,2945,953,2628,2097,890,2139,2421,2290,1115,2588,327,241,1917,2991,2573,19,3911,872,2863,929,839,3893,2178,3822,378,1178,2599,3416,2961,611,3113,2597,2586,161,1429,742,1625,1187,1787,22,3640,3756,776,1724,198,3950]
        y= [939,96,1671,334,666,965,1482,525,1432,1325,1832,1698,169,1533,1945,862,1946,1240,673,1676,1700,857,1711,1420,946,24,1825,962,1498,1079,61,1728,192,1528,1969,1670,598,1513,821,236,1743,280,1830,337,1830,1646,1368,1318,955,474,1183,923,825,135,1622,268,1479,981,1846,1806,1007,1810,1052,302,265,341,687,792,599,674,1673,1559,558,1766,620,102,1619,899,1048,100,901,143,1605,1384,885,1830,1286,906,134,1025,1651,706,1009,987,43,882,392,1642,1810,1558]
    end
    
        #     print(len(x))
    nodes = []
    for i in 1:length(x)
        push!(nodes,Node(x[i],y[i]))
    end
    
    dist_graph=[[Float64(0) for i in 1:length(x)] for i in 1:length(x)]
#     nodes
#     print(length(dist_graph))
#     print(dist_graph[1][3])
    for i in 1:length(x)
        for j in 1:length(x)
            x1=nodes[i].x
            x2=nodes[j].x
            y1=nodes[i].y
            y2=nodes[j].y
            dist = sqrt((x1-x2)^2 + (y1-y2)^2)
            dist_graph[i][j]=dist
        end
    end
    return dist_graph,nodes
 end

create_graph (generic function with 1 method)

In [6]:
       
function obj_function(path,graph)
#     print(path)
    fitness=0
    for i in 1:length(path)-1
#         print(i)
        fitness+=graph[path[i]][path[i+1]]
    end
    fitness+= graph[path[1]][path[end]]
    return fitness
    end

obj_function (generic function with 1 method)

In [7]:
function update_phromone(tau,colony)

    for i in 1:colony.size
        ant=colony.ants[i]
        for j in 2:length(ant.tour)
            tau[ant.tour[j-1],ant.tour[j]] =  tau[ant.tour[j-1],ant.tour[j]]  + 1/ ant.fitness
            tau[ant.tour[j],ant.tour[j-1]] =  tau[ant.tour[j],ant.tour[j-1]]  + 1/ ant.fitness
        end
    end
end


update_phromone (generic function with 1 method)

In [8]:
function roulette_wheel(prob)
    arr=[]
    for i in 1:length(prob)
        push!(arr,[i,prob[i]])
    end
    sorted_array = sort(arr,by=x->x[2],rev=true)
    cum_sum =ones(length(arr))
    sum=1
    for i in 1:length(arr)
        cum_sum[i]=sum
        sum-=sorted_array[i][2]
    end
    r=rand()
    idx=length(arr)
    for i in 2:length(arr)
        if r>cum_sum[i]
            idx=i-1
            break
        end
    end
    return Int64(sorted_array[idx][1])
        
end

roulette_wheel (generic function with 1 method)

In [9]:

    
function createColony( graph, colony , antNo, tau, eta, alpha,  beta)
    node_len = length(graph)
    colony=Colony([],antNo)

    for i in 1:antNo
        tour=[]
        append!(tour,rand(1:node_len))
        for j in 2:node_len
            
            current_node = tour[j-1]
            p_all_nodes =  tau[current_node,:].^alpha .* eta[current_node,:].^beta
            p_all_nodes[tour].=0
            prob = p_all_nodes/sum(p_all_nodes)

            next_node = roulette_wheel(prob)

            append!(tour,next_node)
        end

        tour1 = copy(tour)

        append!(tour1,tour1[1])

        ant = Ant(tour1,obj_function(copy(tour),graph))
        push!(colony.ants,ant)

    end
        
    return colony
end
        
    

createColony (generic function with 1 method)

# with 100 nodes

In [15]:
graph,nodes = create_graph(4)


maxIter = 1000
antNo = 100




# tau_initial = 10 * 1 / (len(nodes) *  np.mean(graph))

tau = ones((length(nodes),length(nodes)))
eta = ones((length(nodes),length(nodes)))

for i in 1:length(graph)
    for j in 1:length(graph)
        eta[i,j]=1/graph[i][j]
    end
end
rho=0.05
max_rho=0.6

min_rho=0.05
alpha=1
beta =4



best_fitness=Inf
best_tour=[]
values=[]
max_values=[]
each_iter_result=[]
for i in 1:maxIter
 

    
    colony = createColony( graph, colony , antNo, tau, eta, alpha,  beta)
    best_ant_in_colony = sort(colony.ants,by = x-> x.fitness)[1]
#     print(best_ant_in_colony.fitness)
    if best_fitness> best_ant_in_colony.fitness
        best_fitness= best_ant_in_colony.fitness
        best_tour=best_ant_in_colony.tour
    end

    update_phromone(tau,colony)
    append!(values,best_fitness)
    append!(max_values,sort(colony.ants,by= x-> x.fitness,rev=true)[1].fitness)
    append!(each_iter_result,best_ant_in_colony.fitness)
    
#     rho = max_rho - i * ( (max_rho - min_rho) / (maxIter))
    tau  = ( 1 - rho ) * tau
end



print(best_fitness,best_tour)
plot(1:maxIter,values)
plot!(1:maxIter,each_iter_result)
plot!(1:maxIter,max_values)


4


LoadError: UndefVarError: colony not defined

# with 50 nodes

In [17]:
graph,nodes = create_graph(3)


maxIter = 1000
antNo = 100




# tau_initial = 10 * 1 / (len(nodes) *  np.mean(graph))

tau = ones((length(nodes),length(nodes)))
eta = ones((length(nodes),length(nodes)))

for i in 1:length(graph)
    for j in 1:length(graph)
        eta[i,j]=1/graph[i][j]
    end
end
rho=0.05
max_rho=0.6

min_rho=0.05
alpha=1
beta =4



best_fitness=Inf
best_tour=[]
values=[]
max_values=[]
each_iter_result=[]
for i in 1:maxIter
 

    
    colony = createColony( graph, colony , antNo, tau, eta, alpha,  beta)
    best_ant_in_colony = sort(colony.ants,by = x-> x.fitness)[1]
#     print(best_ant_in_colony.fitness)
    if best_fitness> best_ant_in_colony.fitness
        best_fitness= best_ant_in_colony.fitness
        best_tour=best_ant_in_colony.tour
    end

    update_phromone(tau,colony)
    append!(values,best_fitness)
    append!(max_values,sort(colony.ants,by= x-> x.fitness,rev=true)[1].fitness)
    append!(each_iter_result,best_ant_in_colony.fitness)
    
#     rho = max_rho - i * ( (max_rho - min_rho) / (maxIter))
    tau  = ( 1 - rho ) * tau
end



print(best_fitness,best_tour)
plot(1:maxIter,values)
plot!(1:maxIter,each_iter_result)
plot!(1:maxIter,max_values)


3


LoadError: UndefVarError: colony not defined

# with 14 nodes

In [12]:
graph,nodes = create_graph(2)


maxIter = 1000
antNo = 70




# tau_initial = 10 * 1 / (len(nodes) *  np.mean(graph))

tau = ones((length(nodes),length(nodes)))
eta = ones((length(nodes),length(nodes)))

for i in 1:length(graph)
    for j in 1:length(graph)
        eta[i,j]=1/graph[i][j]
    end
end
rho=0.05
max_rho=0.6

min_rho=0.05
alpha=1
beta =4



best_fitness=Inf
best_tour=[]
values=[]
max_values=[]
each_iter_result=[]
for i in tqdm(1:maxIter)
 

    
    colony = createColony( graph, colony , antNo, tau, eta, alpha,  beta)
    best_ant_in_colony = sort(colony.ants,by = x-> x.fitness)[1]
#     print(best_ant_in_colony.fitness)
    if best_fitness> best_ant_in_colony.fitness
        best_fitness= best_ant_in_colony.fitness
        best_tour=best_ant_in_colony.tour
    end

    update_phromone(tau,colony)
    append!(values,best_fitness)
    append!(max_values,sort(colony.ants,by= x-> x.fitness,rev=true)[1].fitness)
    append!(each_iter_result,best_ant_in_colony.fitness)
    
    rho = max_rho - i * ( (max_rho - min_rho) / (maxIter))
    tau  = ( 1 - rho ) * tau
end



print(best_fitness,best_tour)
plot(1:maxIter,values)
plot!(1:maxIter,each_iter_result)
plot!(1:maxIter,max_values)


2
0.0%┣                                        ┫ 0/1000 [00:00<00:-50, -20.0 it/s]


LoadError: UndefVarError: colony not defined

In [13]:
graph,nodes = create_graph(4)


maxIter = 1000
antNo = 100




# tau_initial = 10 * 1 / (len(nodes) *  np.mean(graph))

tau = ones((length(nodes),length(nodes)))
eta = ones((length(nodes),length(nodes)))

for i in 1:length(graph)
    for j in 1:length(graph)
        eta[i,j]=1/graph[i][j]
    end
end
rho=0.05
max_rho=0.6

min_rho=0.05
alpha=1
beta =4



best_fitness=Inf
best_tour=[]
values=[]
max_values=[]
each_iter_result=[]
for i in tqdm(1:maxIter)
 

    
    colony = createColony( graph, colony , antNo, tau, eta, alpha,  beta)
    best_ant_in_colony = sort(colony.ants,by = x-> x.fitness)[1]
#     print(best_ant_in_colony.fitness)
    if best_fitness> best_ant_in_colony.fitness
        best_fitness= best_ant_in_colony.fitness
        best_tour=best_ant_in_colony.tour
    end

    update_phromone(tau,colony)
    append!(values,best_fitness)
    append!(max_values,sort(colony.ants,by= x-> x.fitness,rev=true)[1].fitness)
    append!(each_iter_result,best_ant_in_colony.fitness)
    
    rho = max_rho - i * ( (max_rho - min_rho) / (maxIter))
    tau  = ( 1 - rho ) * tau
end



print(best_fitness,best_tour)
plot(1:maxIter,values)
plot!(1:maxIter,each_iter_result)
plot!(1:maxIter,max_values)


4
0.0%┣                                        ┫ 0/1000 [00:00<00:-50, -20.0 it/s]


LoadError: UndefVarError: colony not defined

In [18]:
graph,nodes = create_graph(3)


maxIter = 1000
antNo = 100




# tau_initial = 10 * 1 / (len(nodes) *  np.mean(graph))

tau = ones((length(nodes),length(nodes)))
eta = ones((length(nodes),length(nodes)))

for i in 1:length(graph)
    for j in 1:length(graph)
        eta[i,j]=1/graph[i][j]
    end
end
rho=0.05
max_rho=0.6

min_rho=0.05
alpha=1
beta =4



best_fitness=Inf
best_tour=[]
values=[]
max_values=[]
each_iter_result=[]
for i in tqdm(1:maxIter)
 

    
    colony = createColony( graph, colony , antNo, tau, eta, alpha,  beta)
    best_ant_in_colony = sort(colony.ants,by = x-> x.fitness)[1]
#     print(best_ant_in_colony.fitness)
    if best_fitness> best_ant_in_colony.fitness
        best_fitness= best_ant_in_colony.fitness
        best_tour=best_ant_in_colony.tour
    end

    update_phromone(tau,colony)
    append!(values,best_fitness)
    append!(max_values,sort(colony.ants,by= x-> x.fitness,rev=true)[1].fitness)
    append!(each_iter_result,best_ant_in_colony.fitness)
    
#     rho = max_rho - i * ( (max_rho - min_rho) / (maxIter))
    tau  = ( 1 - rho ) * tau
end



print(best_fitness,best_tour)
plot(1:maxIter,values)
plot!(1:maxIter,each_iter_result)
plot!(1:maxIter,max_values)


3
0.0%┣                                        ┫ 0/1000 [00:00<00:-50, -20.0 it/s]


LoadError: UndefVarError: colony not defined